This script will generate input file for chamber box model. 

In [2]:
import os, importlib, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LinearRegression

sys.path.insert(0, '../../code/src')
#importlib.reload(sys.modules['get_model_input'])
from get_model_input import replace_CTM_by_measurement
from get_model_input import change_format_saprc
from get_model_input import save_saprc_txt

In [3]:
# get temperature
df_temp_pasadena = pd.read_csv('../../data/final/temp_morning_daily_avg_pasadena.csv')
df_temp_redlands = pd.read_csv('../../data/final/temp_morning_daily_avg_redlands.csv')
df_temp_pasadena['Date'] = pd.to_datetime(df_temp_pasadena['Date'])
df_temp_redlands['Date'] = pd.to_datetime(df_temp_redlands['Date'])

## Update CTM data

#### Pasadena

In [4]:
# get measured SAPRC data (NOAA measurement is only availble between 8/7/2021 ~ 9/6/2021)
df_saprc_pasadena = pd.read_csv('../../data/intermediate/saprc11_initial_conc_pasadena.csv')
df_saprc_pasadena['Date'] = pd.to_datetime(df_saprc_pasadena['Date'])
df_saprc_pasadena = df_saprc_pasadena[(df_saprc_pasadena['Date']>='2021-08-07')&
                                      (df_saprc_pasadena['Date']<='2021-09-06')]

In [5]:
## generate input SAPRC file for Pasadena
list_date_pasadena = df_saprc_pasadena['Date'].dt.date.unique()

for date in list_date_pasadena:
    date = pd.to_datetime(date)
    date_2020 = date + pd.offsets.DateOffset(days=1) # date for 2020 CTM data
    if date_2020.day<2 or date_2020.day>30: 
        print('skip ', date) 
    else:  
        # open daily airshed data file
        filepath_CTM = '../../data/raw/CTM_2020/pasadena_4km_2020' + \
            date_2020.strftime('%m%d') + '_11.txt'
        df_CTM = pd.read_csv(filepath_CTM, skiprows=2, delim_whitespace=True)
        df_CTM['target'] = df_CTM['target'].str.strip() # delete any space in str

        # get daily measured saprc, replace CTM by measurement
        df_saprc_daily = df_saprc_pasadena[df_saprc_pasadena['Date']==date].\
            reset_index(drop=True)  
        df_saprc_daily[df_saprc_daily['S11']=='ACET']['Value']
        df_saprc_new = replace_CTM_by_measurement(df_CTM, df_saprc_daily, 'pasadena')

        # correct H2O2   
        if(date == pd.to_datetime('2021-09-01')):
            temp = 21
        else:
            temp = df_temp_pasadena.loc[df_temp_pasadena['Date']==date, 'Value'].values[0]
        df_saprc_new.loc[df_saprc_new['target']=='HO2H','grid'] = \
            np.exp(temp*0.07-1.79)/1000

        df_saprc_new_format = change_format_saprc(df_saprc_new)
        save_saprc_txt(df_saprc_new_format, date, 'Pasadena')

skip  2021-08-30 00:00:00
skip  2021-08-31 00:00:00


#### Redlands

In [161]:
df_saprc_redlands = pd.read_csv('../../data/intermediate/saprc11_initial_conc_redlands.csv')
df_saprc_redlands['Date'] = pd.to_datetime(df_saprc_redlands['Date'])

df_CO_redlands = pd.read_csv('../../data/final/CO_morning_daily_redlands.csv')
df_CO_redlands['Date'] = pd.to_datetime(df_CO_redlands['Date'])

df_PTR_redlands = pd.read_csv('../../data/final/CARB_PTR_redlands.csv')
df_PTR_redlands['dates'] = pd.to_datetime(df_PTR_redlands['dates'])

In [177]:
## generate input SAPRC file for Redlands
list_date_redlands = pd.to_datetime(df_saprc_redlands['Date'].unique())

for date in list_date_redlands:
    date_2020 = date + pd.offsets.DateOffset(days=1) # date for 2020 CTM data
    if date_2020.day<2 or date_2020.day>30: 
        print('skip ', date) 
    else:  
        # open daily airshed data file
        filepath_CTM = '../../data/raw/CTM_2020/redlands_4km_2020' + \
            date_2020.strftime('%m%d') + '_11.txt'
        df_CTM = pd.read_csv(filepath_CTM, skiprows=2, delim_whitespace=True)
        df_CTM['target'] = df_CTM['target'].str.strip() # delete any space in str

        # get daily measured saprc, replace CTM by measurement
        df_saprc_daily = df_saprc_redlands[df_saprc_redlands['Date']==date].\
            reset_index(drop=True)  
        df_saprc_daily[df_saprc_daily['S11']=='ACET']['Value']
        df_saprc_new = replace_CTM_by_measurement(df_CTM, df_saprc_daily, 'redlands')

        # correct CO
        CO = df_CO_redlands.loc[df_CO_redlands['Date']==date, 'Value'].values[0]
        df_saprc_new.loc[df_saprc_new['target']=='CO', 'grid'] = CO

        # constrain Ethene and acetylene by CO
        df_saprc_new.loc[df_saprc_new['target']=='ETHENE', 'grid'] = \
            np.exp(CO*1000*0.01-3.15)/1000
        df_saprc_new.loc[df_saprc_new['target']=='ACET', 'grid'] = \
            np.exp(CO*1000*0.01-3.36)/1000

        # constrain ALK1 and ALK2 by ALK3
        alk3 = df_saprc_new.loc[df_saprc_new['target']=='ALK3', 'grid'].values[0]
        df_saprc_new.loc[df_saprc_new['target']=='ALK1', 'grid'] = \
            (alk3*1000*1.87+0.36)/1000
        df_saprc_new.loc[df_saprc_new['target']=='ALK2', 'grid'] = \
            (alk3*1000*0.37+0.36)/1000

        # correct H2O2 and MACR by temperature
        if date == pd.to_datetime('2021-09-10'):
            temp=33 
        else:           
            temp = df_temp_redlands.loc[df_temp_redlands['Date']==date, 'Value'].values[0]
        df_saprc_new.loc[df_saprc_new['target']=='HO2H','grid'] = \
            np.exp(temp*0.06-1.68)/1000
        df_saprc_new.loc[df_saprc_new['target']=='MACR','grid'] = \
            np.exp(temp*0.11-5.35)/1000

        # add PTR-MS measured VOC
        if date in df_PTR_redlands['dates']:
            df_saprc_new.loc[df_saprc_new['target']=='ISOPRENE','grid'] = \
                df_PTR_redlands.loc[df_PTR_redlands['dates']==date, 'Isoprene'].values[0]/1000
            df_saprc_new.loc[df_saprc_new['target']=='TERP','grid'] = \
                df_PTR_redlands.loc[df_PTR_redlands['dates']==date, 'a-Pinene'].values[0]/1000*10
            df_saprc_new.loc[df_saprc_new['target']=='MVK','grid'] = \
                df_PTR_redlands.loc[df_PTR_redlands['dates']==date, 'MVK'].values[0]/1000
            df_saprc_new.loc[df_saprc_new['target']=='MEK','grid'] = \
                df_PTR_redlands.loc[df_PTR_redlands['dates']==date, 'MEK'].values[0]/1000
            df_saprc_new.loc[df_saprc_new['target']=='ACET','grid'] = \
                df_PTR_redlands.loc[df_PTR_redlands['dates']==date, 'Acetone'].values[0]/1000
        # if no PTR-MS available use temperature to estimate
        else:
            df_saprc_new.loc[df_saprc_new['target']=='ISOPRENE','grid'] = \
                np.exp(temp*0.1-3.52)/1000
            df_saprc_new.loc[df_saprc_new['target']=='TERP','grid'] = \
                np.exp(temp*0.06-3.83)/1000*10
            df_saprc_new.loc[df_saprc_new['target']=='MVK','grid'] = \
                np.exp(temp*0.11-4.57)/1000
            df_saprc_new.loc[df_saprc_new['target']=='MEK','grid'] = \
                df_saprc_new.loc[df_saprc_new['target']=='MVK',
                                 'grid'].iloc[0]*1.17+0.28/1000

        df_saprc_new_format = change_format_saprc(df_saprc_new)
        save_saprc_txt(df_saprc_new_format, date, 'redlands')
        print('saved for ', date)

saved for  2021-07-10 00:00:00
saved for  2021-07-11 00:00:00
saved for  2021-07-12 00:00:00
saved for  2021-07-13 00:00:00
saved for  2021-07-14 00:00:00
saved for  2021-07-15 00:00:00
saved for  2021-07-16 00:00:00
saved for  2021-07-17 00:00:00
saved for  2021-07-18 00:00:00
saved for  2021-07-19 00:00:00
saved for  2021-07-20 00:00:00
saved for  2021-07-21 00:00:00
saved for  2021-07-22 00:00:00
saved for  2021-07-23 00:00:00
saved for  2021-07-24 00:00:00
saved for  2021-07-25 00:00:00
saved for  2021-07-26 00:00:00
saved for  2021-07-27 00:00:00
saved for  2021-07-28 00:00:00
saved for  2021-07-29 00:00:00
skip  2021-07-30 00:00:00
skip  2021-07-31 00:00:00
saved for  2021-08-01 00:00:00
saved for  2021-08-02 00:00:00
saved for  2021-08-03 00:00:00
saved for  2021-08-04 00:00:00
saved for  2021-08-05 00:00:00
saved for  2021-08-06 00:00:00
saved for  2021-08-07 00:00:00
saved for  2021-08-08 00:00:00
saved for  2021-08-09 00:00:00
saved for  2021-08-10 00:00:00
saved for  2021-08

## Setup ambient O3, NO, NO2 data

#### Pasadena

#### Redlands